# Prepare dataset

- Create dataset from the new dataframe

## Librairies

In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 11.5 MB 26.2 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 3.1 MB 25.9 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 84.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 73.8 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 290 kB 28.5 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 99.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 60.5 MB/s eta 0:00:01
     |████████████████████████████████| 35.3 MB 74.1 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 93.5 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 92.6 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 81.4 MB/s eta 0:00:01
     |███████████████

In [2]:
import os
import pickle

import pandas as pd

import numpy as np
import torch

import transformers
from transformers import BertTokenizer

import datasets
from datasets import concatenate_datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import DatasetDict

In [3]:
print('pandas:\t\t', pd.__version__)
print('transformers:\t', transformers.__version__)
print('datasets:\t', datasets.__version__)

pandas:		 1.3.4
transformers:	 4.12.5
datasets:	 1.15.1


## Load Data

In [4]:
dataset_df = pd.read_pickle("/notebooks/KURI-BERT/data/pe_dataset_w_essay_position_pos_tags_pickle")

In [5]:
dataset_df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split_y,...,is_last_in_para,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,0,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...
5970,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,0,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...
5971,essay402,T14,Premise 1448 1525,playing sport makes children getting healthy a...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,7,0,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: VERB, NOUN, VERB, NOUN, V...",Topic: Children should studying hard or

## Prepare dataframe

In [6]:
dataset_df.columns

Index(['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x',
       'label_ComponentType', 'relation_SupportAttack', 'label_RelationType',
       'label_LinkedNotLinked', 'split_y', 'essay', 'argument_bound_1',
       'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr',
       'total_paras', 'token_count', 'token_count_covering_para',
       'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count',
       'succeeding_tokens_in_sentence_count', 'token_ratio',
       'relative_position_in_para_char', 'is_in_intro',
       'relative_position_in_para_token', 'is_in_conclusion',
       'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para',
       'nr_following_comps_in_para', 'structural_fts_as_text',
       'structural_fts_as_text_combined', 'para_ratio', 'first_or_last',
       'strct_fts_w_position_in_essay', 'component_pos_tags',
       'strct_fts_essay_position_pos_tags'],
      dtype='object')

In [7]:
# sanity check
print(len(dataset_df))
dataset_df = dataset_df.dropna()
print(len(dataset_df))

5973
5973


In [8]:
# we need dataset by paragraph

In [9]:
dataset_df.paragraph

0       It is always said that competition can effecti...
1       Consequently, no matter from the view of indiv...
2       First of all, through cooperation, children ca...
3       First of all, through cooperation, children ca...
4       First of all, through cooperation, children ca...
                              ...                        
5968    At the same time, playing sports will give goo...
5969    At the same time, playing sports will give goo...
5970    At the same time, playing sports will give goo...
5971    At the same time, playing sports will give goo...
5972    At the same time, playing sports will give goo...
Name: paragraph, Length: 5973, dtype: object

In [10]:
dataset_df.paragraph.unique()

array(["It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.",
       "Consequently, no matter from the view of individual development or the relationship between competition and cooperation we can receive the same conclusion that a more cooperative attitudes towards life is more profitable in one's success.",
       'First of all, through cooperation, children can learn about interpersonal skills which are significant in the future life of all students. What we acquired from team work is not only how to achieve the same goal

In [11]:
len(dataset_df.paragraph.unique())

1719

In [12]:
paras_df = dataset_df.groupby("paragraph", sort=False)

In [13]:
paras_df

In [14]:
grouped_lists = paras_df["text"].apply(list)

In [15]:
grouped_lists

paragraph
It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.                                                                                                                                                                                                                                                                                           [we should attach more importance to cooperati...
Consequently, no matter from the view of individual development or the relationship between competition and cooperation we can receive t

In [16]:
grouped_lists = grouped_lists.reset_index()

In [40]:
len(grouped_lists['text'][2])

4

In [18]:
grouped_lists_labels = paras_df["label_ComponentType"].apply(list)

In [19]:
grouped_lists_labels

paragraph
It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.                                                                                                                                                                                                                                                                                                                                [MajorClaim]
Consequently, no matter from the view of individual development or the relationship between competition and cooperation we can receive t

In [20]:
grouped_lists_labels = grouped_lists_labels.reset_index()

In [21]:
grouped_lists_labels = grouped_lists_labels.reset_index()

In [22]:
grouped_lists_labels

,index,paragraph,label_ComponentType
0,0,It is always said that competition can effecti...,[MajorClaim]
1,1,"Consequently, no matter from the view of indiv...",[MajorClaim]
2,2,"First of all, through cooperation, children ca...","[Claim, Premise, Premise, Premise]"
3,3,"On the other hand, the significance of competi...","[Claim, Premise, Premise, Premise, Claim]"
4,4,The last 50 years have seen an increasing numb...,[MajorClaim]
...,...,...,...
1714,1714,The second reason why I believe that fatherood...,"[Premise, Premise, Claim, Premise, Premise]"
1715,1715,Some people believe that studying hard is esse...,[MajorClaim]
1716,1716,"In conclusion, both studying hard at school an...",[MajorClaim]
1717,1717,"On the other hand, studying hard will give chi...","[Premise, Premise, Premise, Premise, Claim]"


In [23]:
# Nice.

In [24]:
with open("am_list.txt") as file_am:
    am_list = []
    for line in file_am:
        am_list.append(line.rstrip('\n'))

In [25]:
def am_exists_check(x, am_list):
    
    am = []
    
    sentence = x.sentence
    component = x.text
    
    prefix = sentence[:sentence.find(component)]
    
    am_list = [word.casefold() for word in am_list]
    am_list = list(set(am_list))
    
    for am_word in am_list:
        
        if am_word.casefold() in prefix.casefold() and len(am) == 0:
            am.append(am_word)
    
    return am         

In [26]:
dataset_df['am'] = dataset_df.apply(lambda x: am_exists_check(x, am_list), axis=1)

In [27]:
dataset_df['am']

0       [from this point of view]
1                  [consequently]
2                  [first of all]
3                              []
4                              []
                  ...            
5968                    [because]
5969                           []
5970                         [so]
5971                   [secondly]
5972                           []
Name: am, Length: 5973, dtype: object

In [28]:
# this is correct.

In [29]:
dataset_df['am'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[]                           3130
[so]                          251
[because]                     223
[however]                     201
[for example]                 183
                             ... 
[the reason is that]            2
[from this point of view]       1
[similarly]                     1
[all things considered]         1
[i totally agree]               1
Name: am, Length: 63, dtype: int64

In [30]:
def get_length(x):
    
    am = x.am
    
    return len(am)

In [31]:
dataset_df['am_len'] = dataset_df.apply(lambda x: get_length(x), axis=1)

In [32]:
dataset_df['am_len']

0       1
1       1
2       1
3       0
4       0
       ..
5968    1
5969    0
5970    1
5971    1
5972    0
Name: am_len, Length: 5973, dtype: int64

In [33]:
dataset_df['am_len'].value_counts()

0    3130
1    2843
Name: am_len, dtype: int64

In [34]:
# ok among multiple, we select the first from ones with multiple AMs

In [141]:
dataset_by_paras = dataset_df.groupby(by=['paragraph'], sort=False, as_index=False)

In [144]:
dataset_by_paras.first()

,paragraph,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,...,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,am_flag,am_span
0,It is always said that competition can effecti...,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,...,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,1,[]
1,"Consequently, no matter from the view of indiv...",essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,...,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,1,[]
2,"First of all, through cooperation, children ca...",essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,...,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,1,[]
3,"On the other hand, the significance of competi...",essay001,T7,Claim 1332 1376,competition makes the society more effective,Claim,Claim,[],Attack,Linked,...,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.75,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: NOUN, VERB, DET, NOUN, AD...",Topic: Should students be taught to compete or...,1,"[(0, 0)]"
4,The last 50 years have seen an increasing numb...,essay002,T1,MajorClaim 391 489,they are able to sustain their cultural identi...,MajorClaim,MajorClaim,[],,Linked,...,0,Topic: More people are migrating to other coun...,Topic: More people are migrating to other coun...,0.25,1,Topic: More people are migrating to other coun...,"Part Of Speech tags: PRON, AUX, ADJ, PART, VER...",Topic: More people are migrating to other coun...,1,"[(0, 0)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,The second reason why I believe that fatherood...,essay401,T6,Premise 954 1087,we are living in a society that puts a signifi...,Premise,Premise,[],Support,Linked,...,3,Topic: Fatherhood should be as present as moth...,Topic: Fatherhood should be as present as moth...,0.75,0,Topic: Fatherhood should be as present as moth...,"Part Of Speech tags: PRON, AUX, VERB, ADP, DET...",Topic: Fatherhood should be as present as moth...,1,[]
1715,Some people believe that studying hard is esse...,essay402,T1,MajorClaim 238 307,both of studying hard and playing sports are p...,MajorClaim,MajorClaim,[],,Linked,...,0,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.25,1,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, ADP, VERB, ADJ, CCON...",Topic: Children should studying hard or playin...,1,[]
1716,"In conclusion, both studying hard at school an...",essay402,T2,MajorClaim 1543 1620,both studying hard at school and playing sport...,MajorClaim,MajorClaim,[],,Linked,...,0,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,1.00,1,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, ADV, ADP, NOUN...",Topic: Children should studying hard or playin...,1,[]
1717,"On the other hand, studying hard will gi

In [14]:
dataset_df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split_y,...,is_last_in_para,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,0,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...
5970,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,0,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...
5971,essay402,T14,Premise 1448 1525,playing sport makes children getting healthy a...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,7,0,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: VERB, NOUN, VERB, NOUN, V...",Topic: Children should studying hard or

In [15]:
dataset_df.sentence

0       From this point of view, I firmly believe that...
1       Consequently, no matter from the view of indiv...
2       First of all, through cooperation, children ca...
3       What we acquired from team work is not only ho...
4       During the process of cooperation, children ca...
                              ...                        
5968    It will be good for children, because indirect...
5969    That will make children getting lots of friend...
5970    That will make children getting lots of friend...
5971    Secondly, playing sport makes children getting...
5972    At the same time, playing sports will give goo...
Name: sentence, Length: 5973, dtype: object

In [16]:
# try to see how many sentences have AM from am_list

In [19]:
with open("am_list.txt") as file_am:
    am_list = []
    for line in file_am:
        am_list.append(line.rstrip('\n'))

In [20]:
am_list

['Additionally',
 'As a matter of fact',
 'because',
 'Besides',
 'due to',
 'Finally',
 'First of all',
 'Firstly',
 'for example',
 'For example',
 'For instance',
 'for instance',
 'Furthermore,',
 'has proved it',
 'In addition',
 'In addition to this',
 'In the first place',
 'is due to the fact that',
 'It should also be noted',
 'Moreover',
 'On one hand',
 'On the one hand',
 'On the other hand',
 'One of the main reasons',
 'Secondly',
 'Similarly',
 'since',
 'Since',
 'So',
 'The reason',
 'To begin with',
 'To offer an instance',
 'What is more',
 'As a result',
 'As the consequence',
 'Because',
 'Clearly',
 'Consequently',
 'Considering this subject',
 'Furthermore',
 'Hence',
 'leading to the consequence',
 'so',
 'So',
 'taking account on this fact',
 'That is the reason why',
 'The reason is that',
 'Therefore,',
 'therefore',
 'This means that',
 'This shows that',
 'This will result',
 'Thus',
 'thus',
 'Thus, it is clearly seen that',
 'Thus, it is seen',
 'Thus, th

In [21]:
len(am_list)

115

In [62]:
def am_exists_check(x):
    
    am_flag = 0
    
    sentence = x.sentence
    component = x.text
    
    prefix = sentence[:sentence.find(component)]
    
    for am_word in am_list:
        
        if am_word.casefold() in prefix.casefold():
            am_flag = 1
    
    return am_flag            

In [63]:
dataset_df['am_flag'] = dataset_df.apply(lambda x: am_exists_check(x), axis=1)

In [64]:
dataset_df['am_flag'].value_counts()

0    3130
1    2843
Name: am_flag, dtype: int64

In [47]:
dataset_df['am_flag'][5968]

1

In [48]:
dataset_df['sentence'][5968], dataset_df['text'][5968]

('It will be good for children, because indirectly they will learn how to socialize each other as a team',
 'indirectly they will learn how to socialize each other as a team')

In [87]:
dataset_df['am_flag'][0]

1

In [86]:
dataset_df['sentence'][0], dataset_df['text'][0]

('From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.',
 'we should attach more importance to cooperation during primary education')

In [117]:
def get_am_span(x):
    
    span_l = []
    
    sentence = x.sentence
    component = x.text
    
    prefix = sentence[:sentence.find(component)]
    
    for am_word in am_list:
        
        if am_word.casefold() in prefix.casefold():
            
            prefix_words_list = prefix.casefold().split(" ")
            print(prefix_words_list)
            am_words_list = am_word.casefold().split(" ")
            print(am_words_list)
            span_l = [(i, i+len(am_words_list)-1) for i in range(len(prefix_words_list)) if prefix_words_list[i:i+len(am_words_list)] == am_words_list]
            print(span_l)
            # break
            
        else:
            
            span_l = [(-1, -1)]
    
    return span_l

In [118]:
get_am_span(dataset_df.iloc[0])

['from', 'this', 'point', 'of', 'view,', 'i', 'firmly', 'believe', 'that', '']
['from', 'this', 'point', 'of', 'view']
[]
['from', 'this', 'point', 'of', 'view,', 'i', 'firmly', 'believe', 'that', '']
['i', 'firmly', 'believe', 'that']
[(5, 8)]


[(-1, -1)]

In [106]:
dataset_df['am_span'] = dataset_df.apply(lambda x: get_am_span(x), axis=1)

In [107]:
dataset_df['am_span']

0               []
1               []
2               []
3       [(-1, -1)]
4       [(-1, -1)]
           ...    
5968      [(6, 6)]
5969    [(-1, -1)]
5970      [(8, 8)]
5971            []
5972    [(-1, -1)]
Name: am_span, Length: 5973, dtype: object

In [120]:
a = ['from', 'this', 'point', 'of', 'view,', 'i', 'firmly', 'believe', 'that', '']
b = ['from', 'this', 'point', 'of', 'view']

In [125]:
[(i, i+len(b)) for i in range(len(a)) if a[i:i+len(b)] == b]

[]

In [123]:
len(a)

10

In [124]:
len(b)

5

In [127]:
x = range(len(a))

In [128]:
x

range(0, 10)

In [132]:
indexes = []
for i in range(len(a)):
    print(a[i:i+len(b)])
    if a[i:i+len(b)] == b:
        indexes.append((i, i+len(b)))

['from', 'this', 'point', 'of', 'view,']
['this', 'point', 'of', 'view,', 'i']
['point', 'of', 'view,', 'i', 'firmly']
['of', 'view,', 'i', 'firmly', 'believe']
['view,', 'i', 'firmly', 'believe', 'that']
['i', 'firmly', 'believe', 'that', '']
['firmly', 'believe', 'that', '']
['believe', 'that', '']
['that', '']
['']


In [131]:
indexes

[]

In [ ]:
# include the commas in the argumetn markers
# update the am_list in the file.

In [74]:
a = [2, 5, 6, 7, 0, 2, 19]

In [75]:
b = [0, 2]

In [76]:
z = [(i, i+len(b)-1) for i in range(len(a)) if a[i:i+len(b)] == b]

In [77]:
z[0]

(4, 5)

In [22]:
s = 'hello i am umer'
x = 'i am umer'

In [23]:
s.find(x)

6

In [25]:
s[:s.find(x)]

'hello '

In [8]:
dataset_df = dataset_df.reset_index(drop=True)

In [9]:
dataset_df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split_y,...,is_last_in_para,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,0,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...
5970,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,0,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...
5971,essay402,T14,Premise 1448 1525,playing sport makes children getting healthy a...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,7,0,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: VERB, NOUN, VERB, NOUN, V...",Topic: Children should studying hard or

In [10]:
dataset_df = dataset_df.rename(columns={"split_y": "split"})

In [11]:
dataset_df.columns

Index(['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x',
       'label_ComponentType', 'relation_SupportAttack', 'label_RelationType',
       'label_LinkedNotLinked', 'split', 'essay', 'argument_bound_1',
       'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr',
       'total_paras', 'token_count', 'token_count_covering_para',
       'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count',
       'succeeding_tokens_in_sentence_count', 'token_ratio',
       'relative_position_in_para_char', 'is_in_intro',
       'relative_position_in_para_token', 'is_in_conclusion',
       'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para',
       'nr_following_comps_in_para', 'structural_fts_as_text',
       'structural_fts_as_text_combined', 'para_ratio', 'first_or_last',
       'strct_fts_w_position_in_essay', 'component_pos_tags',
       'strct_fts_essay_position_pos_tags'],
      dtype='object')

In [14]:
dataset_df['paragraph'][0]

"It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education."

In [16]:
dataset_df['paragraph'][1]

"Consequently, no matter from the view of individual development or the relationship between competition and cooperation we can receive the same conclusion that a more cooperative attitudes towards life is more profitable in one's success."

In [23]:
dataset_df['sentence'][0]

'From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.'

In [24]:
dataset_df['sentence'][1]

"Consequently, no matter from the view of individual development or the relationship between competition and cooperation we can receive the same conclusion that a more cooperative attitudes towards life is more profitable in one's success."

In [26]:
dataset_df['text'][0]

'we should attach more importance to cooperation during primary education'

In [27]:
dataset_df['text'][1]

"a more cooperative attitudes towards life is more profitable in one's success"

In [28]:
dataset_df['paragraph'][2]

'First of all, through cooperation, children can learn about interpersonal skills which are significant in the future life of all students. What we acquired from team work is not only how to achieve the same goal with others but more importantly, how to get along with others. During the process of cooperation, children can learn about how to listen to opinions of others, how to communicate with others, how to think comprehensively, and even how to compromise with other team members when conflicts occurred. All of these skills help them to get on well with other people and will benefit them for the whole life.'

In [29]:
dataset_df['sentence'][2]

'First of all, through cooperation, children can learn about interpersonal skills which are significant in the future life of all students'

In [30]:
dataset_df['text'][2]

'through cooperation, children can learn about interpersonal skills which are significant in the future life of all students'

In [31]:
dataset_df['text'][3]

'What we acquired from team work is not only how to achieve the same goal with others but more importantly, how to get along with others'

In [32]:
dataset_df['text'][4]

'During the process of cooperation, children can learn about how to listen to opinions of others, how to communicate with others, how to think comprehensively, and even how to compromise with other team members when conflicts occurred'

In [33]:
dataset_df['text'][5]

'All of these skills help them to get on well with other people and will benefit them for the whole life'

## train_test_split

In [32]:
train_df = dataset_df[dataset_df.split=='TRAIN'].reset_index(drop=True)
test_df = dataset_df[dataset_df.split=='TEST'].reset_index(drop=True)



# for example

# val_df = dataset_df[dataset_df.split=='VALIDATION'].reset_index(drop=True)

## Prepare Dataset object

In [33]:
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

In [34]:
train_val_datasets = dataset_train.train_test_split(train_size=0.8)
dataset_train = train_val_datasets['train']
dataset_val = train_val_datasets['test']

In [35]:
dataset = DatasetDict({"train": dataset_train, "test": dataset_test, "validation": dataset_val})

In [36]:
dataset

DatasetDict({
    train: Dataset({
        features: ['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x', 'label_ComponentType', 'relation_SupportAttack', 'label_RelationType', 'label_LinkedNotLinked', 'split', 'essay', 'argument_bound_1', 'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr', 'total_paras', 'token_count', 'token_count_covering_para', 'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count', 'succeeding_tokens_in_sentence_count', 'token_ratio', 'relative_position_in_para_char', 'is_in_intro', 'relative_position_in_para_token', 'is_in_conclusion', 'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para', 'nr_following_comps_in_para', 'structural_fts_as_text', 'structural_fts_as_text_combined', 'para_ratio', 'first_or_last', 'strct_fts_w_position_in_essay', 'component_pos_tags', 'strct_fts_essay_position_pos_tags', 'strct_fts_essay_position_combined', 'strct_fts_essay_position_pos_tagscombined', 'ner_list', 'ft

In [37]:
# sanity check
set(dataset['train']['split']), set(dataset['test']['split']), set(dataset['validation']['split'])

({'TRAIN'}, {'TEST'}, {'TRAIN'})

## Save dataset

In [38]:
torch.save(dataset, os.path.join("/notebooks/linguistic_features/data/hf_datasets", 'pe_dataset_w_ner_entities.pt'))